In [35]:
import gspread


'''Подключение таблицы с подготовленными данными
    в client_secrets.json содержится данные для подключения к проекту с подготовленными данными Google cloud
'''
gs = gspread.service_account(filename='service_account.json') # Подключение к сервису
sh_clean = gs.open_by_key('1yuCSKE7hbbX_ChOu9vCFJVdnAkC3T9rSg9pFpZIulOA') # Инициализация таблицы
worksheet_clean= sh_clean.sheet1 #Указание листа
clean_data = worksheet_clean.get_all_values() #Получение данных


In [36]:
import pandas as pd


df_graph = pd.DataFrame(clean_data[1:], columns=clean_data[0])


In [37]:
df_graph['x'] = df_graph['x'].str.replace(',', '.') # Замена запятых на точки в колонке 'x'
df_graph['y'] = df_graph['y'].str.replace(',', '.') # Замена запятых на точки в колонке 'y'
df_graph['x'] = df_graph['x'].astype(float) # Преобразование в тип float
df_graph['y'] = df_graph['y'].astype(float)# Преобразование в тип float


In [41]:
import os
import matplotlib.pyplot as plt
import textwrap
from adjustText import adjust_text
from view_upload import upload_photo

# Параметры для графиков и текста
text_fontsize = 15 # Размер текста 'keyword'
marker_size = 200 # Размер точек распределения 
circle_size = 100 # Размер точек на легенде
edge_color = 'black' # Цвет обводки точек
edge_linewidth = 1 # Толщина обводки точек
footer_fontsize = 20 # Размер текста footer-подписи
label_fontsize = 15 # Размер текста легенды 
title_fontsize = 17 # # Размер названия легенды 


unique_areas = df_graph['area'].unique() # Уникальные значения 'area' 

for area in unique_areas: # Цикл для создания графиков для каждой области
    area_df = df_graph[df_graph['area'] == area] # Выбор данных по текущей области
    width = 1500 / 80  # Ширина в дюймах (по условию ТЗ)
    height = 1500 / 80  # Высота в дюймах (по условию ТЗ)

    fig, ax = plt.subplots(figsize=(width, height)) # Создание объектов графика и осей
    scatter_handles = []

    # Добавление точек на график с определенными параметрами
    ax.scatter(area_df['x'], area_df['y'], c=area_df['color'], marker='o', s=circle_size, edgecolors=edge_color, linewidths=edge_linewidth)

    # Отбор уникальных значений 'cluster_name', 'color' для легенды
    labels_df = area_df.drop_duplicates(subset='cluster_name')[['cluster_name', 'color']]

    text_labels = []
    # Добавление текстовых меток с оберткой на график
    for i, keyword in enumerate(area_df['keyword']): # перебор всех элементов в колонке 'keyword'
        wrapped_keyword = "\n".join(textwrap.wrap(keyword, width=15)) # перенос если 'keyword' > 15 символов
        # Параметры выставления текста 'keyword' над точками масштаб, прозрачность текста, выравнивание
        text_labels.append(ax.text(area_df['x'].iloc[i], area_df['y'].iloc[i], wrapped_keyword,
                                   fontsize=text_fontsize, alpha=0.7, ha='center', va='center'))

    # Создание точек для элементов легенды
    for _, row in labels_df.iterrows():
        # Создание  точки с цветом, формой, названием и размером
        scatter = ax.scatter([], [], c=row['color'], marker='o', label=row['cluster_name'], s=marker_size)
        scatter_handles.append(scatter) # Добавление созданной точки в список 

    # Оптимизация расположения текста над точками с помощью библиотеки AdjustText
    adjust_text(text_labels, force_points=(2, 2), arrowprops=dict(arrowstyle='->', color='black', mutation_scale=15, linewidth=1.5))

    # Добавление легенды на график, название легенты, размер текста
    legend = ax.legend(handles=scatter_handles, title='Кластеры', markerscale=1, fontsize=label_fontsize,
                       title_fontsize=title_fontsize, loc='center left', bbox_to_anchor=(1, 0.5))

    # Добавление footer-подписи внизу графика 
    footer_text = f'Диаграмма рассеивания для области {area}' # Формирование текста
    ax.text(0.6, -0.05, footer_text, transform=ax.transAxes, ha='center', va='center', fontsize=footer_fontsize) # расположение и размер текста

    # Удаление подписей на осях и рамки (по примеру svg файла из ТЗ)
    ax.set_xticks([]) # Удаление подписи на оси х
    ax.set_yticks([]) # Удаление подписи на оси у
    ax.axis('off') # Удаление рамки 

    # Сохранение графиков в файл и его загрузка
    folder_path = r'result_graph' # Директория проекта для хранения графиков
    name_graph = f'{area}_scatter_graph.png' # формирование названия графика
    file_path = os.path.join(folder_path, name_graph) # соеденение в путь для сохранения
    plt.savefig(file_path, bbox_inches='tight') # Сохранение локально
    upload_photo(file_path, name_graph) # Сохранение в Google drive

    plt.close()# Закрытие текущего графика
